In [9]:
# Import required libraries
from binance import Client
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from get_data import get_historical_data
from datetime import datetime, timedelta
from pickle import load

In [2]:
# Initialize the variables
window_size = 100
prediction_window = 48

In [3]:
# Import model .pkl file and use it to predict the price for the next 48 time steps.
model = load_model('bitcoin_model.h5')
scaler = load(open('bitcoin_scaler.pkl', 'rb'))

In [12]:
# Get the historical data for the last 100 time steps
today = datetime.today() - timedelta(days=2)
end_date = today.strftime('%Y-%m-%d')
start_date = (today - timedelta(days=5)).strftime('%Y-%m-%d')
df = pd.read_csv('historical_data_30_min.csv')
# df = get_historical_data('BTC', start_date, end_date, Client.KLINE_INTERVAL_30MINUTE)
# It looks like this method doesn't give us the today's price candles
time_array = df['close_time'].to_numpy()
price_array = df['close'].to_numpy()
price_array = scaler.fit_transform(price_array.reshape(-1, 1))

In [13]:
d = []
for index in range(len(price_array) - window_size):
    d.append(price_array[index: index + window_size])
d = np.array(d)
d = d.reshape((d.shape[0], d.shape[1], 1))

x_data = d[:, :-1, :]
y_data = d[:, -1, :]

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size=0.1, shuffle=False)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(15655, 99, 1) (1740, 99, 1) (15655, 1) (1740, 1)


In [11]:
y_pred = model.predict(X_test)
# print(mean_squared_error(Y_test, y_pred))
print(np.mean(np.abs(Y_test - y_pred) * 100 / Y_test))

13.951711500520915


In [14]:
window_size = 101
d = []
for index in range(len(price_array) - window_size):
    d.append(price_array[index: index + window_size])
d = np.array(d)
d = d.reshape((d.shape[0], d.shape[1], 1))

x_data = d[:, :-2, :]
y_data = d[:, -1, :]

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size=0.1, shuffle=False)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(15654, 99, 1) (1740, 99, 1) (15654, 1) (1740, 1)


In [16]:
y_pred = model.predict(X_test)  # This is the first step prediction
# Now append this prediction to X_test and make another prediction
X_test = np.append(X_test, y_pred.reshape((y_pred.shape[0], 1, 1)), axis=1)[:, 1:, :]
y_pred = model.predict(X_test)
# print(mean_squared_error(Y_test, y_pred))
print(np.mean(np.abs(Y_test - y_pred) * 100 / Y_test))

20.636016914171986


In [21]:
window_size = 102
d = []
for index in range(len(price_array) - window_size):
    d.append(price_array[index: index + window_size])
d = np.array(d)
d = d.reshape((d.shape[0], d.shape[1], 1))

x_data = d[:, :-3, :]
y_data = d[:, -1, :]

# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size=0.1, shuffle=False)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(15653, 99, 1) (1740, 99, 1) (15653, 1) (1740, 1)


In [22]:
y_pred = model.predict(X_test)  # This is the first step prediction
# Now append this prediction to X_test and make another prediction
X_test = np.append(X_test, y_pred.reshape((y_pred.shape[0], 1, 1)), axis=1)[:, 1:, :]
y_pred = model.predict(X_test)

X_test = np.append(X_test, y_pred.reshape((y_pred.shape[0], 1, 1)), axis=1)[:, 1:, :]
y_pred = model.predict(X_test)

# print(mean_squared_error(Y_test, y_pred))
print(np.mean(np.abs(Y_test - y_pred) * 100 / Y_test))

26.218329112784144
